In [ ]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut
import country_script
import education_script
import health_script
import population_script
import quality_of_life_script

# Data Summarization

In [ ]:
country_df = country_script.getCountryList(True)
fact_table_df = pd.read_csv("./output/fact_table.csv")
year_df = pd.read_csv("./output/year.csv")

In [ ]:
countries = ["Canada", "United States", "United Kingdom", "France", "Germany", "Sweden", "Australia", "Denmark", "Belgium"]

In [ ]:
main_df = pd.merge(country_df, fact_table_df, on='CountryKey' )
main_df = pd.merge(main_df, year_df, on='YearKey')

In [ ]:
main_df["GDP"] = pd.to_numeric(main_df["GDP"])


In [ ]:
gdp_df = main_df.pivot(index="Year", columns="Name", values="GDP")

In [ ]:
gdp_df.plot()
plt.show()

In [ ]:
education_df = pd.read_csv("./output/education.csv")

In [ ]:
main_df = pd.merge(main_df, education_df, left_on="EducationKey", right_on="EducationKey")

In [ ]:
main_df["School enrollment, tertiary (% gross)"] = pd.to_numeric(main_df["School enrollment, tertiary (% gross)"])

In [ ]:
se_df = main_df.pivot(index="Year", columns="Name", values="School enrollment, tertiary (% gross)")

In [ ]:
se_df.plot.bar()
plt.show()

In [ ]:
qualityoflife_df = pd.read_csv("./output/quality_of_life.csv")

In [ ]:
main_df = pd.merge(main_df, qualityoflife_df, left_on="QualityOfLifeKey", right_on="QualityOfLifeKey")

In [ ]:
main_df["Unemployment, total (% of total labor force)"] = pd.to_numeric(main_df["Unemployment, total (% of total labor force)"])

In [ ]:
unemployment_df = main_df.pivot(index="Year", columns="Name", values="Unemployment, total (% of total labor force)")

In [ ]:
unemployment_df.hist(column="Canada")
plt.show()

In [ ]:
main_df["Public spending on education, total (% of GDP)"] = pd.to_numeric(main_df["Public spending on education, total (% of GDP)"])
main_df["PISA: Mean performance on the mathematics scale"] = pd.to_numeric(main_df["PISA: Mean performance on the mathematics scale"])

In [ ]:
pisa_df = main_df.pivot(index="Year", columns="Name", values=["Public spending on education, total (% of GDP)","PISA: Mean performance on the mathematics scale"])

In [ ]:
pisa_df.plot.scatter(x='Public spending on education, total (% of GDP)',y='PISA: Mean performance on the mathematics scale')
plt.show()

In [ ]:
population_df = pd.read_csv("./output/population.csv")

In [ ]:
main_df = pd.merge(main_df, population_df, left_on="PopulationKey", right_on="PopulationKey")

In [ ]:
main_df["Urban population growth (annual %)"] = pd.to_numeric(main_df["Urban population growth (annual %)"])

In [ ]:
upg_df = main_df.pivot(index="Year", columns="Name", values="Urban population growth (annual %)")

In [ ]:
upg_df.plot.box(figsize=(15,15))
plt.show()

# Data Preprocessing
Want to create a model to predict a country's human development index based on attribute values from their health, population, quality of life, and education statistics.

In [ ]:
final_df = pd.read_csv("./output/fact_table.csv")

In [ ]:
# columns with empty values
# education dimension
# literacy rate, adult male, female, total
# school enrollment primary, secondary, teritary
# public spending on education, total
# Pisa science, reading, mathmatics
# gender parity

# health dimension
# domestic government health expenditure
# hospital beds 
# immunization HEPB3, DPT, measles, pol3
# stillbirth rate
# nurses and midwives, physicians
# diabetes prevalence
# prevalence of HIV, tobacco use
# total alcohol consumption per capita
# mortality rate, male and female

# population dimension
# life expectancy at birth: female, male, total
# net migration

# quality of life dimension
# people with basic handashing 
# pregnant women receiving prenatla care
# births attended by skilled health staff
# community health workers
# consumption of iodized salt


In [ ]:
# Country
# wil use - TotalPopulation, Birthrate, Size, GDP, Population Density
# all those columns are full
country_df = country_script.getCountryList(True)
country_df = country_df[["CountryKey", "Name", "TotalPopulation", "Birthrate", "Size", "GDP", "PopulationDensity"]]
final_df = pd.merge(final_df, country_df, left_on="CountryKey", right_on="CountryKey")

In [ ]:
# Education
curent_df = education_script.getEducationList()
# will drop the 3 empty literacy rate columns
curent_df = curent_df.drop(["Literacy rate, adult female (% of females ages 15 and above)",
        "Literacy rate, adult male (% of males ages 15 and above)",
       "Literacy rate, adult total (% of people ages 15 and above)"], axis=1)
# setting the wanted columns
wanted_col_names = ['School enrollment, primary (% gross)',
       'School enrollment, secondary (% gross)',
       'School enrollment, tertiary (% gross)',
       'Public spending on education, total (% of GDP)',
       'PISA: Mean performance on the science scale',
       'PISA: Mean performance on the reading scale',
       'PISA: Mean performance on the mathematics scale',
       'Adjusted net enrolment rate, one year before the official primary entry age, adjusted gender parity index (GPIA)']
for current_col_name in wanted_col_names:
        curent_df[current_col_name] = pd.to_numeric(curent_df[current_col_name])
        for country in countries:
                current_col = curent_df[curent_df["Country Name"] == country][[current_col_name]]
                mean = np.mean(current_col)
                #current_col = current_col.fillna(mean)
                current_col.fillna(mean, inplace=True)
                for i in current_col.index:
                        curent_df.at[i, current_col_name] = current_col.at[i, current_col_name]

curent_df = curent_df[wanted_col_names]
#education_df.to_csv("temp.csv")
final_df = pd.merge(final_df, curent_df, left_on="EducationKey", right_index=True)

In [ ]:
# Health
curent_df = health_script.getHealthList()
#print(health_df.columns)
# drop empty diabetes prevalence column
curent_df = curent_df.drop(['Diabetes prevalence (% of population ages 20 to 79)'], axis=1)
wanted_col_names = [
       'Domestic general government health expenditure (% of GDP)',
       'Hospital beds (per 1,000 people)',
       'Immunization, HepB3 (% of one-year-old children)',
       'Immunization, DPT (% of children ages 12-23 months)',
       'Immunization, measles (% of children ages 12-23 months)',
       'Immunization, Pol3 (% of one-year-old children)',
       'Mortality rate, infant (per 1,000 live births)',
       'Stillbirth rate (per 1,000 total births)',
       'Nurses and midwives (per 1,000 people)',
       'Physicians (per 1,000 people)',
       'Prevalence of HIV, total (% of population ages 15-49)',
       'Prevalence of current tobacco use (% of adults)',
       'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)',
       'Mortality rate, adult, male (per 1,000 male adults)',
       'Mortality rate, adult, female (per 1,000 female adults)']

for current_col_name in wanted_col_names:
        curent_df[current_col_name] = pd.to_numeric(curent_df[current_col_name])
        for country in countries:
                current_col = curent_df[curent_df["Country Name"] == country][[current_col_name]]
                mean = np.mean(current_col)
                #current_col = current_col.fillna(mean)
                current_col.fillna(mean, inplace=True)
                for i in current_col.index:
                        curent_df.at[i, current_col_name] = current_col.at[i, current_col_name]
curent_df = curent_df[wanted_col_names]
#curent_df.to_csv("temp.csv")
final_df = pd.merge(final_df, curent_df, left_on="HealthKey", right_index=True)
# will use - domesic general government health expenditure 

# Hospital beds

# immunization, HepB3

# immunization DPT

# immuzation measles

# immunization pol3

# mortality rate, infant

# stillbirth rate 

# nurses and midwives

# physicians

# drop diabetes prevalence - no data

# prevalence of HIV - lots of missing data

# prevalence of current tobacco - lots of missing data

# total alcohol consumption per capita 

# mortality rate male

# mortality rate female



In [ ]:
# Population
curent_df = population_script.getPopulationList()
# drop net migration
curent_df = curent_df.drop(['Net migration'], axis=1)
wanted_col_names = ['Life expectancy at birth, female (years)',
       'Life expectancy at birth, male (years)',
       'Life expectancy at birth, total (years)',
       'Population growth (annual %)',
       'Labor force, female (% of total labor force)', 'Labor force, total',
       'Rural population growth (annual %)',
       'Urban population growth (annual %)',
       'Rural population (% of total population)',
       'Urban population (% of total population)']

for current_col_name in wanted_col_names:
        curent_df[current_col_name] = pd.to_numeric(curent_df[current_col_name])
        for country in countries:
                current_col = curent_df[curent_df["Country Name"] == country][[current_col_name]]
                mean = np.mean(current_col)
                #current_col = current_col.fillna(mean)
                current_col.fillna(mean, inplace=True)
                for i in current_col.index:
                        curent_df.at[i, current_col_name] = current_col.at[i, current_col_name]

curent_df = curent_df[wanted_col_names]
#curent_df.to_csv("temp.csv")
final_df = pd.merge(final_df, curent_df, left_on="PopulationKey", right_index=True)
# life expectancy at birth female

# life expectancy at birth male

# life expectancy at birth total

# net migration - might drop, little data

# population growth 

# labor force female

# labor force total

# rural population growth

# urban population grwoth

# rural population 

# urban population

In [ ]:
# Quality of life
curent_df = quality_of_life_script.getQualityOfLifeList()
# drop empty columns
curent_df = curent_df.drop(['People with basic handwashing facilities including soap and water (% of population)',
                            'Pregnant women receiving prenatal care (%)',
                            'Community health workers (per 1,000 people)',
                            'Consumption of iodized salt (% of households)'
                            ], axis=1)
wanted_col_names = ['People using at least basic drinking water services (% of population)',
       'People using at least basic sanitation services (% of population)',
       'Unemployment, female (% of female labor force)',
       'Unemployment, male (% of male labor force)',
       'Unemployment, total (% of total labor force)',
       'Age dependency ratio (% of working-age population)',
       'Births attended by skilled health staff (% of total)']

for current_col_name in wanted_col_names:
        curent_df[current_col_name] = pd.to_numeric(curent_df[current_col_name])
        for country in countries:
                current_col = curent_df[curent_df["Country Name"] == country][[current_col_name]]
                mean = np.mean(current_col)
                #current_col = current_col.fillna(mean)
                current_col.fillna(mean, inplace=True)
                for i in current_col.index:
                        curent_df.at[i, current_col_name] = current_col.at[i, current_col_name]

curent_df = curent_df[wanted_col_names]
#curent_df.to_csv("temp.csv")
final_df = pd.merge(final_df, curent_df, left_on="QualityOfLifeKey", right_index=True)
# people using at least basic drinking water services

# people using at least basic sanitation services

# people with basic handwashing facilities including soap - drop, no data

# unemployment female

# unemployment male

# unemployment total

# pregnant women receiving prenatal care - drop, no data

# age dependency ratio 

# births attended by skilled health staff - little data

# community health workers - drop, no data

# consumption of iodized salt - drop, no data

In [ ]:
final_df.to_csv("final.csv")
print(final_df.columns)


In [ ]:
# columns to scale
columns_to_scale = [
       'TotalPopulation', 'Birthrate', 'Size', 'GDP', 'PopulationDensity',
       'School enrollment, primary (% gross)',
       'School enrollment, secondary (% gross)',
       'School enrollment, tertiary (% gross)',
       'Public spending on education, total (% of GDP)',
       'PISA: Mean performance on the science scale',
       'PISA: Mean performance on the reading scale',
       'PISA: Mean performance on the mathematics scale',
       'Adjusted net enrolment rate, one year before the official primary entry age, adjusted gender parity index (GPIA)',
       'Domestic general government health expenditure (% of GDP)',
       'Hospital beds (per 1,000 people)',
       'Immunization, HepB3 (% of one-year-old children)',
       'Immunization, DPT (% of children ages 12-23 months)',
       'Immunization, measles (% of children ages 12-23 months)',
       'Immunization, Pol3 (% of one-year-old children)',
       'Mortality rate, infant (per 1,000 live births)',
       'Stillbirth rate (per 1,000 total births)',
       'Nurses and midwives (per 1,000 people)',
       'Physicians (per 1,000 people)',
       'Prevalence of HIV, total (% of population ages 15-49)',
       'Prevalence of current tobacco use (% of adults)',
       'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)',
       'Mortality rate, adult, male (per 1,000 male adults)',
       'Mortality rate, adult, female (per 1,000 female adults)',
       'Life expectancy at birth, female (years)',
       'Life expectancy at birth, male (years)',
       'Life expectancy at birth, total (years)',
       'Population growth (annual %)',
       'Labor force, female (% of total labor force)', 'Labor force, total',
       'Rural population growth (annual %)',
       'Urban population growth (annual %)',
       'Rural population (% of total population)',
       'Urban population (% of total population)',
       'People using at least basic drinking water services (% of population)',
       'People using at least basic sanitation services (% of population)',
       'Unemployment, female (% of female labor force)',
       'Unemployment, male (% of male labor force)',
       'Unemployment, total (% of total labor force)',
       'Age dependency ratio (% of working-age population)',
       'Births attended by skilled health staff (% of total)']
scaler = MinMaxScaler()
final_df[columns_to_scale] = scaler.fit_transform(final_df[columns_to_scale])
final_df.to_csv("final.csv")

In [ ]:
Y = final_df[~final_df["DevelopmentIndex"].isna()]["DevelopmentIndex"]
X = final_df.iloc[Y.index]
X = X[columns_to_scale]
X = X.drop(X.columns[X.isna().any()].tolist(), axis=1)
select = SelectKBest(chi2, k=20)
X_new = select.fit_transform(X, Y)
#selected_support = select.get_feature_names_out()
selected_support = select.get_support(indices=True)
selected_features = X.columns[selected_support]
print(selected_features)

In [ ]:
X = X[selected_features]
X.to_csv("X.csv")

# Classification

In [ ]:
model_dtree = DecisionTreeClassifier(random_state=42)
model_gboosting = GradientBoostingClassifier(random_state=42)
model_rforest = RandomForestClassifier(random_state=42)

In [ ]:
# Decision Tree Classifier
decision_tree_params = {
        'max_depth': [i for i in range(3,25)], 'min_samples_split': [i for i in range(2,5)], 'min_samples_leaf': [i for i in range(4,10)]
}
scoring = {"accuracy": "accuracy", "precision": "precision", "recall": "recall", "bal_accuracy": "balanced_accuracy"}
# need to set the parameters grid
# defaults 5 fold cross validation
decision_tree_clf = GridSearchCV(model_dtree, param_grid=decision_tree_params, scoring=scoring, cv=4, refit='bal_accuracy')
decision_tree_clf.fit(X, Y)
print(decision_tree_clf.best_estimator_)
#print(decision_tree_clf.best_index_)

In [ ]:
#sorted(decision_tree_clf.cv_results_.keys())
best_index = decision_tree_clf.best_index_
print(decision_tree_clf.best_params_)
scores_wanted = ['mean_test_accuracy', 'mean_test_precision', 'mean_test_recall', 'mean_fit_time']
for s in scores_wanted:
    print(f"{s}: {decision_tree_clf.cv_results_[s][best_index]}")

In [ ]:
# Gradient Boosting Classifier
gradient_boosting_params = {
    'max_depth': [i for i in range(3,25)], 'min_samples_split': [i for i in range(2,5)], 'min_samples_leaf': [i for i in range(4,10)]}
scoring = {"accuracy": "accuracy", "precision": "precision", "recall": "recall", "bal_accuracy": "balanced_accuracy"}
gradient_boosting_clf = GridSearchCV(model_gboosting, param_grid=gradient_boosting_params,scoring=scoring, cv=3, refit='bal_accuracy')
gradient_boosting_clf.fit(X,Y)
#print(gradient_boosting_clf.cv_results_)

In [ ]:
best_index = gradient_boosting_clf.best_index_
print(gradient_boosting_clf.best_params_)
scores_wanted = ['mean_test_accuracy', 'mean_test_precision', 'mean_test_recall', 'mean_fit_time']
for s in scores_wanted:
    print(f"{s}: {gradient_boosting_clf.cv_results_[s][best_index]}")

In [104]:
# Random Forest Classifier
random_forest_params = {'max_depth': [i for i in range(3,25)], 'min_samples_split': [i for i in range(2,5)], 'min_samples_leaf': [i for i in range(4,10)]}
scoring = {"accuracy": "accuracy", "precision": "precision", "recall": "recall", "bal_accuracy": "balanced_accuracy"}
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    random_forest_clf = GridSearchCV(model_rforest, param_grid=random_forest_params, scoring=scoring, cv=4, refit='bal_accuracy')
    random_forest_clf.fit(X,Y)
#print(random_forest_clf.cv_results_)

In [105]:
best_index = random_forest_clf.best_index_
print(random_forest_clf.best_params_)
scores_wanted = ['mean_test_accuracy', 'mean_test_precision', 'mean_test_recall', 'mean_fit_time']
for s in scores_wanted:
    print(f"{s}: {random_forest_clf.cv_results_[s][best_index]}")

{'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}
mean_test_accuracy: 0.7777777777777777
mean_test_precision: 0.8541666666666666
mean_test_recall: 0.625
mean_fit_time: 0.13500118255615234
